##Clustering

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TFM/Dataset y archivos iniciales/datasetSeleccionadasPreClustering.csv', decimal='.')

In [ ]:
import pandas as pd

# Filtrar las columnas necesarias y combinar los equipos locales y visitantes
teams_home = df[['season', 'home_team']].rename(columns={'home_team': 'team'})
teams_away = df[['season', 'away_team']].rename(columns={'away_team': 'team'})

teams_per_season = pd.concat([teams_home, teams_away]).drop_duplicates()

output_path_teams_per_season = '/content/drive/MyDrive/TFM/8. Presupuesto equipos/teams_per_season.csv'

teams_per_season.to_csv(output_path_teams_per_season, index=False)

print(teams_per_season.head())

  season               team
0  10/11          blackpool
1  10/11          liverpool
2  10/11  manchester united
3  10/11         stoke city
4  10/11             fulham


In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/viu/teams_per_season_valor_mercado.csv'
teams_per_season_valor_mercado = pd.read_csv(file_path, delimiter=';')

print(teams_per_season_valor_mercado.columns)

# Eliminar la columna 'season'
if 'season' in teams_per_season_valor_mercado.columns:
    teams_per_season_valor_mercado = teams_per_season_valor_mercado.drop(columns=['season'])

# Renombrar la columna 'valor_mercado' a 'valor_mercado_avg'
teams_per_season_valor_mercado = teams_per_season_valor_mercado.rename(columns={'valor_mercado': 'valor_mercado_avg'})

# Calcular la media de 'valor_mercado_avg' para cada equipo
teams_per_season_valor_mercado_avg = teams_per_season_valor_mercado.groupby('team')['valor_mercado_avg'].mean().reset_index()
teams_per_season_valor_mercado_avg['valor_mercado_avg'] = teams_per_season_valor_mercado_avg['valor_mercado_avg'].round(2)

# Ordenar los equipos por valor de mercado promedio de mayor a menor
teams_per_season_valor_mercado_avg = teams_per_season_valor_mercado_avg.sort_values(by='valor_mercado_avg', ascending=False)

output_path_sorted_avg = '/content/drive/MyDrive/TFM/8. Presupuesto equipos/teams_per_season_valor_mercado_avg_sorted.csv'
teams_per_season_valor_mercado_avg.to_csv(output_path_sorted_avg, index=False)

print(teams_per_season_valor_mercado_avg.head())

Index(['season', 'team', 'valor_mercado'], dtype='object')
                 team  valor_mercado_avg
19    manchester city             698.55
10            chelsea             662.09
18          liverpool             571.91
20  manchester united             560.45
31  tottenham hotspur             515.91


In [ ]:
import pandas as pd

teams_per_season_valor_mercado_avg_sorted = pd.read_csv('/content/drive/MyDrive/TFM/8. Presupuesto equipos/teams_per_season_valor_mercado_avg_sorted.csv')

file_path_clustered = '/content/drive/MyDrive/TFM/2. Locales más ganadores por capacidad/clustered_analysis_avg_points_vs_capacity_sorted.csv'
clustered_analysis = pd.read_csv(file_path_clustered)

# Renombrar la columna 'team' a 'home_team'
teams_per_season_valor_mercado_avg_sorted = teams_per_season_valor_mercado_avg_sorted.rename(columns={'team': 'home_team'})

# Fusionar los DataFrames en base a la columna 'home_team'
merged_df = pd.merge(clustered_analysis, teams_per_season_valor_mercado_avg_sorted, on='home_team', how='left')

# Ordenar el DataFrame final por 'avg_points'
merged_df_sorted = merged_df.sort_values(by='avg_points', ascending=False)

output_path_merged = '/content/drive/MyDrive/TFM/8. Presupuesto equipos/merged_clustered_analysis_with_market_value.csv'
merged_df.to_csv(output_path_merged, index=False)

print(merged_df.head())


           home_team  avg_points  games_played  capacity  cluster  \
0    manchester city        2.46           205     55017        2   
1  manchester united        2.17           203     74879        2   
2            chelsea        2.14           203     40853        2   
3            arsenal        2.12           203     60704        2   
4          liverpool        2.10           204     60725        2   

   valor_mercado_avg  
0             698.55  
1             560.45  
2             662.09  
3             487.64  
4             571.91  


In [ ]:
# Calcular estadísticas descriptivas
descriptive_stats = merged_df.describe()
print(descriptive_stats)

       avg_points  games_played      capacity    cluster  valor_mercado_avg
count   37.000000     37.000000     37.000000  37.000000          37.000000
mean     1.422432    110.000000  35198.189189   0.864865         221.824595
std      0.384718     67.510082  14949.482245   0.787477         174.751077
min      0.840000     13.000000  11329.000000   0.000000          46.000000
25%      1.180000     38.000000  25586.000000   0.000000         107.000000
50%      1.340000    109.000000  31800.000000   1.000000         154.500000
75%      1.470000    167.000000  40853.000000   1.000000         238.000000
max      2.460000    205.000000  74879.000000   2.000000         698.550000


In [ ]:
# Calcular la matriz de correlación
correlation_matrix = merged_df[['capacity', 'valor_mercado_avg', 'avg_points', 'games_played']].corr()
print(correlation_matrix)

                   capacity  valor_mercado_avg  avg_points  games_played
capacity           1.000000           0.719698    0.693551      0.659338
valor_mercado_avg  0.719698           1.000000    0.925630      0.717950
avg_points         0.693551           0.925630    1.000000      0.765794
games_played       0.659338           0.717950    0.765794      1.000000


In [ ]:
import statsmodels.api as sm

# Variables independientes y dependientes
X = merged_df[['capacity', 'valor_mercado_avg']]
y = merged_df['avg_points']

X = sm.add_constant(X)

# Ajustar el modelo de regresión lineal
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             avg_points   R-squared:                       0.858
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                     103.0
Date:                Wed, 26 Jun 2024   Prob (F-statistic):           3.72e-15
Time:                        23:35:00   Log-Likelihood:                 19.506
No. Observations:                  37   AIC:                            -33.01
Df Residuals:                      34   BIC:                            -28.18
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.9389      0.06

In [ ]:
# Equipos exitosos
successful_teams = merged_df[merged_df['avg_points'] > merged_df['avg_points'].mean()]
print("Equipos exitosos:\n", successful_teams[['home_team', 'avg_points', 'valor_mercado_avg', 'capacity']])

# Equipos poco exitosos
unsuccessful_teams = merged_df[merged_df['avg_points'] < merged_df['avg_points'].mean()]
print("Equipos poco exitosos:\n", unsuccessful_teams[['home_team', 'avg_points', 'valor_mercado_avg', 'capacity']])

# Equipos sorprendentes
surprising_teams = merged_df[((merged_df['valor_mercado_avg'] < merged_df['valor_mercado_avg'].mean()) &
                              (merged_df['avg_points'] > merged_df['avg_points'].mean())) |
                             ((merged_df['capacity'] < merged_df['capacity'].mean()) &
                              (merged_df['avg_points'] > merged_df['avg_points'].mean()))]

surprising_teams = surprising_teams.head(8)
print("Equipos sorprendentes:\n", surprising_teams[['home_team', 'avg_points', 'valor_mercado_avg', 'capacity']])

print("Equipos exitosos destacados:\n", successful_teams[['home_team', 'avg_points', 'valor_mercado_avg', 'capacity']].head(4))
print("Equipos poco exitosos destacados:\n", unsuccessful_teams[['home_team', 'avg_points', 'valor_mercado_avg', 'capacity']].head(4))
print("Equipos sorprendentes destacados:\n", surprising_teams[['home_team', 'avg_points', 'valor_mercado_avg', 'capacity']])

Equipos exitosos:
             home_team  avg_points  valor_mercado_avg  capacity
0     manchester city        2.46             698.55     55017
1   manchester united        2.17             560.45     74879
2             chelsea        2.14             662.09     40853
3             arsenal        2.12             487.64     60704
4           liverpool        2.10             571.91     60725
5   tottenham hotspur        2.04             515.91     62850
6             everton        1.78             299.09     39571
7     west ham united        1.47             221.90     62500
8    newcastle united        1.45             195.10     52338
9      leicester city        1.67             301.00     32273
10         stoke city        1.55             127.50     30089
11       swansea city        1.43             129.43     21088
Equipos poco exitosos:
                    home_team  avg_points  valor_mercado_avg  capacity
12               southampton        1.41             223.22     3238

## Regresión lineal

In [ ]:
import pandas as pd
import statsmodels.api as sm

# Archivo con los valores de mercado y el rendimiento de los equipos
file_path = '/content/drive/MyDrive/TFM/8. Presupuesto equipos/merged_clustered_analysis_with_market_value.csv'
merged_df = pd.read_csv(file_path)

# Equipos que permanecen en la liga
teams_to_keep = [
    'arsenal', 'aston villa', 'brighton and hove albion', 'burnley', 'chelsea',
    'crystal palace', 'everton', 'leeds united', 'leicester city', 'liverpool',
    'manchester city', 'manchester united', 'newcastle united', 'southampton',
    'tottenham hotspur', 'west ham united', 'wolverhampton wanderers'
]

# Datos para los equipos ascendidos (Norwich City, Watford, Brentford)
new_teams_data = {
    'team': ['norwich city', 'watford', 'brentford'],
    'capacity': [27244, 21577, 17250],
    'valor_mercado_avg': [96.60, 189.60, 130.00],
    'avg_points': [1.26, 1.37, 1.30]
}

new_teams_df = pd.DataFrame(new_teams_data)

# Filtrar los equipos que permanecen en la liga
teams_remaining = merged_df[merged_df['home_team'].isin(teams_to_keep)]
teams_remaining = teams_remaining[['home_team', 'capacity', 'valor_mercado_avg', 'avg_points']]

teams_remaining = teams_remaining.rename(columns={'home_team': 'team'})

# Combinar los datos de los equipos restantes y los equipos ascendidos
all_teams_df = pd.concat([teams_remaining, new_teams_df], ignore_index=True)

# Definir las variables dependientes e independientes
X = all_teams_df[['capacity', 'valor_mercado_avg']]
y = all_teams_df['avg_points']

X = sm.add_constant(X)

# Ajustar el modelo de regresión lineal
model = sm.OLS(y, X).fit()

# Predecir los puntos para la temporada 21/22
all_teams_df['points'] = model.predict(X)

# Calcular points_home como la media de puntos de cada home_team multiplicada por 19
all_teams_df['points_home'] = all_teams_df['avg_points'] * 19

# Ordenar los equipos por puntos en orden descendente para determinar la posición
all_teams_df = all_teams_df.sort_values(by='points', ascending=False).reset_index(drop=True)

# Añadir la columna de posición
all_teams_df['position'] = all_teams_df.index + 1

final_df = all_teams_df[['position', 'team', 'points', 'points_home']]
output_path = '/content/drive/MyDrive/TFM/8. Presupuesto equipos/predicted_premier_league_21_22.csv'
final_df.to_csv(output_path, index=False)

print(final_df.head())


   position               team    points  points_home
0         1    manchester city  2.382208        46.74
1         2            chelsea  2.281927        40.66
2         3  manchester united  2.155625        41.23
3         4          liverpool  2.148356        39.90
4         5  tottenham hotspur  2.044115        38.76


In [ ]:
import pandas as pd

# Archivo con la predicción de puntos
predicted_file_path = '/content/drive/MyDrive/TFM/8. Presupuesto equipos/predicted_premier_league_21_22.csv'
predicted_df = pd.read_csv(predicted_file_path)

real_data = {
    'position': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'team': [
        'liverpool', 'manchester city', 'arsenal', 'tottenham hotspur', 'manchester united', 'chelsea',
        'leicester city', 'west ham united', 'newcastle united', 'crystal palace', 'everton',
        'southampton', 'brentford', 'wolverhampton wanderers', 'aston villa', 'brighton and hove albion',
        'burnley', 'leeds united', 'norwich city', 'watford'
    ],
    'points': [49, 47, 41, 40, 35, 34, 34, 32, 30, 29, 29, 25, 24, 24, 23, 22, 21, 18, 12, 8]
}

real_df = pd.DataFrame(real_data)

# Comparar los datos predichos con los reales
comparison_df = pd.merge(predicted_df, real_df, on='team', suffixes=('_predicted', '_real'))

# Calcular la diferencia entre puntos predichos y reales
comparison_df['points_difference'] = comparison_df['points_home'] - comparison_df['points_real']

print(comparison_df)


    position_predicted                      team  points_predicted  \
0                    1           manchester city          2.382208   
1                    2                   chelsea          2.281927   
2                    3         manchester united          2.155625   
3                    4                 liverpool          2.148356   
4                    5         tottenham hotspur          2.044115   
5                    6                   arsenal          1.984782   
6                    7                   everton          1.574834   
7                    8            leicester city          1.563326   
8                    9           west ham united          1.472843   
9                   10   wolverhampton wanderers          1.458071   
10                  11              leeds united          1.452781   
11                  12               southampton          1.412621   
12                  13          newcastle united          1.399652   
13                  

In [ ]:
import pandas as pd
import statsmodels.api as sm

# Archivo con los valores de mercado y el rendimiento de los equipos
file_path = '/content/drive/MyDrive/TFM/8. Presupuesto equipos/merged_clustered_analysis_with_market_value.csv'
merged_df = pd.read_csv(file_path)

# Equipos que permanecen en la liga
teams_to_keep = [
    'arsenal', 'aston villa', 'brighton and hove albion', 'burnley', 'chelsea',
    'crystal palace', 'everton', 'leeds united', 'leicester city', 'liverpool',
    'manchester city', 'manchester united', 'newcastle united', 'southampton',
    'tottenham hotspur', 'west ham united', 'wolverhampton wanderers'
]

# Datos para los equipos ascendidos (Norwich City, Watford, Brentford)
new_teams_data = {
    'team': ['norwich city', 'watford', 'brentford'],
    'capacity': [27244, 21577, 17250],
    'avg_points': [1.26, 1.37, 1.30]
}

new_teams_df = pd.DataFrame(new_teams_data)

# Filtrar los equipos que permanecen en la liga
teams_remaining = merged_df[merged_df['home_team'].isin(teams_to_keep)]
teams_remaining = teams_remaining[['home_team', 'capacity', 'avg_points']]

teams_remaining = teams_remaining.rename(columns={'home_team': 'team'})

# Combinar los datos de los equipos restantes y los equipos ascendidos
all_teams_df = pd.concat([teams_remaining, new_teams_df], ignore_index=True)

# Definir las variables dependientes e independientes
X = all_teams_df[['capacity']]
y = all_teams_df['avg_points']

X = sm.add_constant(X)

# Modelo de regresión lineal
model = sm.OLS(y, X).fit()

# Predecir los puntos para la temporada 21/22
all_teams_df['points'] = model.predict(X)

# Calcular points_home como la media de puntos de cada home_team multiplicada por 19
all_teams_df['points_home'] = all_teams_df['avg_points'] * 19

# Ordenar los equipos por puntos en orden descendente para determinar la posición
all_teams_df = all_teams_df.sort_values(by='points', ascending=False).reset_index(drop=True)

# Añadir la columna de posición
all_teams_df['position'] = all_teams_df.index + 1

final_df = all_teams_df[['position', 'team', 'points', 'points_home']]
output_path = '/content/drive/MyDrive/TFM/8. Presupuesto equipos/predicted_premier_league_21_22_capacity_only.csv'
final_df.to_csv(output_path, index=False)

print(final_df.head())


   position               team    points  points_home
0         1  manchester united  2.188397        41.23
1         2  tottenham hotspur  1.980580        38.76
2         3    west ham united  1.974533        27.93
3         4          liverpool  1.943868        39.90
4         5            arsenal  1.943505        40.28


In [ ]:
import pandas as pd

# Archivo con la predicción de puntos
predicted_file_path = '/content/drive/MyDrive/TFM/8. Presupuesto equipos/predicted_premier_league_21_22_capacity_only.csv'
predicted_df = pd.read_csv(predicted_file_path)

real_data = {
    'position': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'team': [
        'liverpool', 'manchester city', 'arsenal', 'tottenham hotspur', 'manchester united', 'chelsea',
        'leicester city', 'west ham united', 'newcastle united', 'crystal palace', 'everton',
        'southampton', 'brentford', 'wolverhampton wanderers', 'aston villa', 'brighton and hove albion',
        'burnley', 'leeds united', 'norwich city', 'watford'
    ],
    'points': [49, 47, 41, 40, 35, 34, 34, 32, 30, 29, 29, 25, 24, 24, 23, 22, 21, 18, 12, 8]
}

real_df = pd.DataFrame(real_data)

# Comparar los datos predichos con los reales
comparison_df = pd.merge(predicted_df, real_df, on='team', suffixes=('_predicted', '_real'))

# Calcular la diferencia entre puntos predichos y reales
comparison_df['points_difference'] = comparison_df['points_home'] - comparison_df['points_real']

print(comparison_df)


    position_predicted                      team  points_predicted  \
0                    1         manchester united          2.188397   
1                    2         tottenham hotspur          1.980580   
2                    3           west ham united          1.974533   
3                    4                 liverpool          1.943868   
4                    5                   arsenal          1.943505   
5                    6           manchester city          1.845254   
6                    7          newcastle united          1.798971   
7                    8               aston villa          1.632151   
8                    9                   chelsea          1.600552   
9                   10                   everton          1.578404   
10                  11              leeds united          1.549362   
11                  12               southampton          1.454239   
12                  13            leicester city          1.452321   
13                  

In [ ]:
import pandas as pd
import statsmodels.api as sm

# Archivo con los valores de mercado y el rendimiento de los equipos
file_path = '/content/drive/MyDrive/TFM/8. Presupuesto equipos/merged_clustered_analysis_with_market_value.csv'
merged_df = pd.read_csv(file_path)

# Equipos que permanecen en la liga
teams_to_keep = [
    'arsenal', 'aston villa', 'brighton and hove albion', 'burnley', 'chelsea',
    'crystal palace', 'everton', 'leeds united', 'leicester city', 'liverpool',
    'manchester city', 'manchester united', 'newcastle united', 'southampton',
    'tottenham hotspur', 'west ham united', 'wolverhampton wanderers'
]

# Datos para los equipos ascendidos (Norwich City, Watford, Brentford)
new_teams_data = {
    'team': ['norwich city', 'watford', 'brentford'],
    'valor_mercado_avg': [120.0, 100.0, 50.0],
    'avg_points': [1.26, 1.37, 1.30]
}

new_teams_df = pd.DataFrame(new_teams_data)

# Filtrar los equipos que permanecen en la liga
teams_remaining = merged_df[merged_df['home_team'].isin(teams_to_keep)]
teams_remaining = teams_remaining[['home_team', 'valor_mercado_avg', 'avg_points']]

teams_remaining = teams_remaining.rename(columns={'home_team': 'team'})

# Combinar los datos de los equipos restantes y los equipos ascendidos
all_teams_df = pd.concat([teams_remaining, new_teams_df], ignore_index=True)

# Definir las variables dependientes e independientes
X = all_teams_df[['valor_mercado_avg']]
y = all_teams_df['avg_points']

X = sm.add_constant(X)

# Modelo de regresión lineal
model = sm.OLS(y, X).fit()

# Predecir los puntos para la temporada 21/22
all_teams_df['points'] = model.predict(X)

# Calcular points_home como la media de puntos de cada home_team multiplicada por 19
all_teams_df['points_home'] = all_teams_df['avg_points'] * 19

# Ordenar los equipos por puntos en orden descendente para determinar la posición
all_teams_df = all_teams_df.sort_values(by='points', ascending=False).reset_index(drop=True)

# Añadir la columna de posición
all_teams_df['position'] = all_teams_df.index + 1

final_df = all_teams_df[['position', 'team', 'points', 'points_home']]
output_path = '/content/drive/MyDrive/TFM/8. Presupuesto equipos/predicted_premier_league_21_22_market_value_only.csv'
final_df.to_csv(output_path, index=False)

print(final_df.head())


   position               team    points  points_home
0         1    manchester city  2.376172        46.74
1         2            chelsea  2.304699        40.66
2         3          liverpool  2.127918        39.90
3         4  manchester united  2.105452        41.23
4         5  tottenham hotspur  2.018140        38.76


In [ ]:
import pandas as pd

# Archivo con la predicción de puntos
predicted_file_path = '/content/drive/MyDrive/TFM/8. Presupuesto equipos/predicted_premier_league_21_22_market_value_only.csv'
predicted_df = pd.read_csv(predicted_file_path)

real_data = {
    'position': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'team': [
        'liverpool', 'manchester city', 'arsenal', 'tottenham hotspur', 'manchester united', 'chelsea',
        'leicester city', 'west ham united', 'newcastle united', 'crystal palace', 'everton',
        'southampton', 'brentford', 'wolverhampton wanderers', 'aston villa', 'brighton and hove albion',
        'burnley', 'leeds united', 'norwich city', 'watford'
    ],
    'points': [49, 47, 41, 40, 35, 34, 34, 32, 30, 29, 29, 25, 24, 24, 23, 22, 21, 18, 12, 8]
}

real_df = pd.DataFrame(real_data)

# Comparar los datos predichos con los reales
comparison_df = pd.merge(predicted_df, real_df, on='team', suffixes=('_predicted', '_real'))

# Calcular la diferencia entre puntos predichos y reales
comparison_df['points_difference'] = comparison_df['points_home'] - comparison_df['points_real']

print(comparison_df)

    position_predicted                      team  points_predicted  \
0                    1           manchester city          2.376172   
1                    2                   chelsea          2.304699   
2                    3                 liverpool          2.127918   
3                    4         manchester united          2.105452   
4                    5         tottenham hotspur          2.018140   
5                    6                   arsenal          1.962722   
6                    7            leicester city          1.596848   
7                    8                   everton          1.593104   
8                    9   wolverhampton wanderers          1.490991   
9                   10              leeds united          1.473348   
10                  11               southampton          1.444375   
11                  12           west ham united          1.441787   
12                  13  brighton and hove albion          1.415029   
13                  

In [ ]:
#R² (Coeficiente de Determinación)

# Ajuste del modelo basado en valor de mercado
X_market = sm.add_constant(merged_df['valor_mercado_avg'])
model_market = sm.OLS(merged_df['avg_points'], X_market).fit()

# Ajuste del modelo basado en capacidad del estadio
X_capacity = sm.add_constant(merged_df['capacity'])
model_capacity = sm.OLS(merged_df['avg_points'], X_capacity).fit()

# Evaluar R²
r2_market = model_market.rsquared
r2_capacity = model_capacity.rsquared

print("R² del modelo basado en valor de mercado:", r2_market)
print("R² del modelo basado en capacidad del estadio:", r2_capacity)


R² del modelo basado en valor de mercado: 0.8567914710152569
R² del modelo basado en capacidad del estadio: 0.4810123200015636


In [ ]:
#RMSE (Root Mean Square Error)

from sklearn.metrics import mean_squared_error
import numpy as np

# Predicciones del modelo basado en valor de mercado
y_pred_market = model_market.predict(X_market)
rmse_market = np.sqrt(mean_squared_error(merged_df['avg_points'], y_pred_market))

# Predicciones del modelo basado en capacidad del estadio
y_pred_capacity = model_capacity.predict(X_capacity)
rmse_capacity = np.sqrt(mean_squared_error(merged_df['avg_points'], y_pred_capacity))

print("RMSE del modelo basado en valor de mercado:", rmse_market)
print("RMSE del modelo basado en capacidad del estadio:", rmse_capacity)


RMSE del modelo basado en valor de mercado: 0.14360750710999592
RMSE del modelo basado en capacidad del estadio: 0.27338282418268994


In [ ]:
#Validación Cruzada

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

X_market_cv = merged_df[['valor_mercado_avg']]
X_capacity_cv = merged_df[['capacity']]
y_cv = merged_df['avg_points']

# Modelo de regresión lineal
lr = LinearRegression()

# Validación cruzada para el modelo basado en valor de mercado
scores_market = cross_val_score(lr, X_market_cv, y_cv, cv=5, scoring='r2')
mean_score_market = np.mean(scores_market)

# Validación cruzada para el modelo basado en capacidad del estadio
scores_capacity = cross_val_score(lr, X_capacity_cv, y_cv, cv=5, scoring='r2')
mean_score_capacity = np.mean(scores_capacity)

print("R² promedio del modelo basado en valor de mercado (validación cruzada):", mean_score_market)
print("R² promedio del modelo basado en capacidad del estadio (validación cruzada):", mean_score_capacity)


R² promedio del modelo basado en valor de mercado (validación cruzada): -2.863311834926816
R² promedio del modelo basado en capacidad del estadio (validación cruzada): -9.42499351043111
